In [1]:
from hasp.make_feature_pipeline import make_feature_pipeline
from sklearn.pipeline import Pipeline

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## Data

In [2]:
from us8kdata.loader import UrbanSound8K
data = UrbanSound8K('../hasp/data')


In [3]:
train_fold = [samples for samples in data.fold_audio_generator(fold=[1, 2])]#, 3, 6, 7, 8, 9, 10])]
val_fold = [samples for samples in data.fold_audio_generator(fold=[4])]
test_fold = [samples for samples in data.fold_audio_generator(fold=[5])]


In [4]:
train_y = data.filter_metadata(fold=[1, 2]).classID
val_y = data.filter_metadata(fold=[4]).classID
test_y = data.filter_metadata(fold=[5]).classID

## Training Pipeline

In [5]:
pipe = make_feature_pipeline()


In [6]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier by deafult criterion='gini'

In [7]:
full_pipe = Pipeline([
    ('preproc', pipe),
    ('rand_forest', RandomForestClassifier())
])

In [8]:
full_pipe.fit(train_fold, train_y)

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('mean_mfcc',
                                  FunctionTransformer(func=<function samples_to_mean_mfcc at 0x110dab790>,
                                                      kw_args={'fmax': None,
                                                               'fmin': 0.0,
                                                               'hop_length': 128,
                                                               'n_fft': 512,
                                                               'n_mfcc': 20,
                                                               'sr': 16000})),
                                 ('scaler', StandardScaler())])),
                ('rand_forest', RandomForestClassifier())])

In [9]:
pred_y = full_pipe.predict(val_fold)

## Performance

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
print(classification_report(val_y, pred_y))

              precision    recall  f1-score   support

           0       0.71      0.12      0.21       100
           1       0.83      0.17      0.28        59
           2       0.22      0.35      0.27       100
           3       0.33      0.74      0.46       100
           4       0.49      0.55      0.52       100
           5       0.55      0.24      0.34       107
           6       0.86      0.32      0.46        38
           7       0.39      0.30      0.34       120
           8       0.66      0.35      0.46       166
           9       0.24      0.55      0.34       100

    accuracy                           0.38       990
   macro avg       0.53      0.37      0.37       990
weighted avg       0.50      0.38      0.37       990



In [12]:
print(confusion_matrix(val_y, pred_y))

[[12  0 38  0 26  3  0  0  3 18]
 [ 0 10  4  5  4  2  0  6  1 27]
 [ 0  0 35 30  7  0  1  5  1 21]
 [ 0  0  8 74  8  0  1  1  3  5]
 [ 0  0  6  6 55  1  0 27  0  5]
 [ 3  0 23 12  0 26  0 14 18 11]
 [ 0  0  6  9  0  0 12  3  0  8]
 [ 1  0 13  0  3 10  0 36  2 55]
 [ 0  0  8 69  7  2  0  0 58 22]
 [ 1  2 15 20  2  3  0  0  2 55]]


## Comparing with model of different criterion (When criterion='entropy')

In [68]:
full_pipe1 = Pipeline([
    ('preproc', pipe),
    ('model', RandomForestClassifier(criterion='entropy'))
])

In [69]:
full_pipe1.fit(train_fold, train_y)

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('mean_mfcc',
                                  FunctionTransformer(func=<function samples_to_mean_mfcc at 0x110dab790>,
                                                      kw_args={'fmax': None,
                                                               'fmin': 0.0,
                                                               'hop_length': 128,
                                                               'n_fft': 512,
                                                               'n_mfcc': 20,
                                                               'sr': 16000})),
                                 ('scaler', StandardScaler())])),
                ('model', RandomForestClassifier(criterion='entropy'))])

In [70]:
pred_y1 = full_pipe1.predict(val_fold)

In [72]:
print(classification_report(val_y, pred_y1))

              precision    recall  f1-score   support

           0       0.53      0.09      0.15       100
           1       0.53      0.15      0.24        59
           2       0.21      0.32      0.25       100
           3       0.35      0.77      0.48       100
           4       0.53      0.58      0.55       100
           5       0.53      0.29      0.37       107
           6       0.79      0.39      0.53        38
           7       0.46      0.31      0.37       120
           8       0.72      0.39      0.51       166
           9       0.24      0.54      0.33       100

    accuracy                           0.39       990
   macro avg       0.49      0.38      0.38       990
weighted avg       0.48      0.39      0.38       990



In [73]:
print(confusion_matrix(val_y, pred_y1))

[[ 9  0 39  2 24  1  0  0  2 23]
 [ 1  9  7  5  6  1  0  4  1 25]
 [ 1  1 32 31 12  1  2  3  1 16]
 [ 0  0  8 77  6  0  2  0  4  3]
 [ 0  5  6  0 58  1  0 22  0  8]
 [ 1  0 25 10  0 31  0 11 16 13]
 [ 0  0  5  8  1  0 15  4  0  5]
 [ 4  0  5  0  1 15  0 37  0 58]
 [ 0  0 14 64  0  5  0  0 65 18]
 [ 1  2 15 21  2  4  0  0  1 54]]


## Other comparison 

When fmin=500, fmax=4000, criterion='gini'

In [28]:
pipe2 = make_feature_pipeline()

In [29]:
pipe2.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'mean_mfcc', 'scaler', 'mean_mfcc__accept_sparse', 'mean_mfcc__check_inverse', 'mean_mfcc__feature_names_out', 'mean_mfcc__func', 'mean_mfcc__inv_kw_args', 'mean_mfcc__inverse_func', 'mean_mfcc__kw_args', 'mean_mfcc__validate', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std'])

In [30]:
pipe2.set_params(mean_mfcc__kw_args={'fmin':500, 'fmax':4000})

Pipeline(steps=[('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x110dab790>,
                                     kw_args={'fmax': 4000, 'fmin': 500})),
                ('scaler', StandardScaler())])

In [31]:
full_pipe2 = Pipeline([
            ('preproc', pipe),
            ('model', RandomForestClassifier())
            ])

In [32]:
full_pipe2.fit(train_fold, train_y)

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('mean_mfcc',
                                  FunctionTransformer(func=<function samples_to_mean_mfcc at 0x110dab790>,
                                                      kw_args={'fmax': None,
                                                               'fmin': 0.0,
                                                               'hop_length': 128,
                                                               'n_fft': 512,
                                                               'n_mfcc': 20,
                                                               'sr': 16000})),
                                 ('scaler', StandardScaler())])),
                ('model', RandomForestClassifier())])

In [33]:
pred_y2 = full_pipe2.predict(val_fold)

In [34]:
print(classification_report(val_y, pred_y2))

              precision    recall  f1-score   support

           0       0.62      0.10      0.17       100
           1       0.82      0.15      0.26        59
           2       0.24      0.36      0.29       100
           3       0.34      0.74      0.47       100
           4       0.54      0.56      0.55       100
           5       0.58      0.29      0.39       107
           6       0.77      0.26      0.39        38
           7       0.38      0.33      0.35       120
           8       0.67      0.35      0.46       166
           9       0.22      0.53      0.31       100

    accuracy                           0.38       990
   macro avg       0.52      0.37      0.36       990
weighted avg       0.50      0.38      0.37       990



In [35]:
print(confusion_matrix(val_y, pred_y2))

[[10  0 35  1 27  2  0  1  3 21]
 [ 0  9  4  5  4  0  0  5  2 30]
 [ 0  0 36 29  6  2  2  4  1 20]
 [ 0  0  8 74  5  0  1  1  4  7]
 [ 0  0  6  4 56  1  0 30  0  3]
 [ 0  0 17 18  0 31  0 19 16  6]
 [ 0  0  7  9  0  0 10  3  0  9]
 [ 2  0  4  1  1  9  0 39  0 64]
 [ 0  0 13 60  4  4  0  0 58 27]
 [ 4  2 19 14  1  4  0  0  3 53]]


## Camparison 3

When fmin=500, fmax=4000, criterion='entropy'

In [36]:
pipe3 = make_feature_pipeline()

In [38]:
pipe3.set_params(mean_mfcc__kw_args={'fmin':500, 'fmax':4000})

Pipeline(steps=[('mean_mfcc',
                 FunctionTransformer(func=<function samples_to_mean_mfcc at 0x110dab790>,
                                     kw_args={'fmax': 4000, 'fmin': 500})),
                ('scaler', StandardScaler())])

In [40]:
full_pipe3 = Pipeline([
            ('preproc', pipe),
            ('model', RandomForestClassifier(criterion='entropy'))
            ])

In [41]:
full_pipe3.fit(train_fold, train_y)

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('mean_mfcc',
                                  FunctionTransformer(func=<function samples_to_mean_mfcc at 0x110dab790>,
                                                      kw_args={'fmax': None,
                                                               'fmin': 0.0,
                                                               'hop_length': 128,
                                                               'n_fft': 512,
                                                               'n_mfcc': 20,
                                                               'sr': 16000})),
                                 ('scaler', StandardScaler())])),
                ('model', RandomForestClassifier(criterion='entropy'))])

In [42]:
pred_y3 = full_pipe3.predict(val_fold)

In [43]:
print(classification_report(val_y, pred_y3))

              precision    recall  f1-score   support

           0       0.69      0.11      0.19       100
           1       0.71      0.20      0.32        59
           2       0.24      0.38      0.30       100
           3       0.33      0.77      0.46       100
           4       0.56      0.58      0.57       100
           5       0.60      0.30      0.40       107
           6       0.88      0.37      0.52        38
           7       0.32      0.23      0.27       120
           8       0.72      0.34      0.47       166
           9       0.25      0.57      0.35       100

    accuracy                           0.39       990
   macro avg       0.53      0.39      0.38       990
weighted avg       0.51      0.39      0.38       990



## Conclusion

In [48]:
import pandas as pd

In [90]:
table = pd.DataFrame(classification_report(val_y, pred_y, output_dict=True)).transpose()
table1 = pd.DataFrame(classification_report(val_y, pred_y1, output_dict=True)).transpose()
table2 = pd.DataFrame(classification_report(val_y, pred_y2, output_dict=True)).transpose()
table3 = pd.DataFrame(classification_report(val_y, pred_y3, output_dict=True)).transpose()

In [84]:
results = {
    'mean_precision': [table[['precision']].mean(), table1[["precision"]].mean(), table2[['precision']].mean(), table3[['precision']].mean()], 
    'mean_recall': [table[['recall']].mean(), table1[['recall']].mean(), table2[['recall']].mean(), table3[['recall']].mean()], 
    'mean_f1score': [table[['f1-score']].mean(), table1[['f1-score']].mean(), table2[['f1-score']].mean(), table3[['f1-score']].mean()]           
}

In [85]:
df = pd.DataFrame(results)
df

,mean_precision,mean_recall,mean_f1score
0,precision 0.514888 dtype: float64,recall 0.369998 dtype: float64,f1-score 0.367638 dtype: float64
1,precision 0.480166 dtype: float64,recall 0.3848 dtype: float64,f1-score 0.380068 dtype: float64
2,precision 0.506846 dtype: float64,recall 0.368953 dtype: float64,f1-score 0.365791 dtype: float64
3,precision 0.517506 dtype: float64,recall 0.386085 dtype: float64,f1-score 0.383661 dtype: float64


## Results

RFC model with criterion='entropy', pipeline params of mean_mfcc__kw_args={'fmin'=500, 'fmax'=4000} has the best overall performance. However, if looking directly onto the precisions, recall, f1-score of each fold, base model has better performance.